# 2024 OIBC Challenge
## (Deep Learning version skeleton 02)

### 데이터 전처리

In [1]:
import pandas as pd
from functools import reduce
import os
from datetime import datetime, timedelta
from tqdm import tqdm
print(os.getcwd())  # 현재 작업 경로 출력

c:\Users\user\Desktop\24-OIBC-WolgoksTears\personal_files\hoon


actual_weather_1

In [2]:
# 데이터 불러오기
actual_weather_1 = pd.read_csv("../../data_files/actual_weather_1.csv")

# 'ts'를 datetime 형식으로 변환 후 1시간 단위로 맞추기
actual_weather_1['datetime'] = pd.to_datetime(actual_weather_1['ts'], unit='s')
actual_weather_1['datetime'] = actual_weather_1['datetime'].dt.round('h') + timedelta(hours=9)

# 원래 'ts' 열과 'location' 제거
actual_weather_1.drop(columns=['ts'], inplace=True)
actual_weather_1.drop(columns=['location'], inplace=True)

# 1시간 단위로 그룹화하여 평균을 계산 (location을 제외한 모든 열)
actual_weather_1 = actual_weather_1.groupby(['datetime'], as_index=False).mean()


# weather_forecast에 없는 columns 제거
actual_weather_1.drop(columns=['real_feel_temp_shade'], inplace=True)
actual_weather_1.drop(columns=['pressure'], inplace=True)
actual_weather_1.drop(columns=['appr_temp'], inplace=True)
actual_weather_1.drop(columns=['wind_chill_temp'], inplace=True)
 
# 결과 확인
actual_weather_1.head()

,datetime,temp,real_feel_temp,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,vis,cld_cvr,ceiling,wet_bulb_temp,precip_1h
0,2024-03-01 00:00:00,8.500000,8.000000,81.000000,5.500000,270.000000,6.100000,17.600000,0.0,20.900000,91.000000,579.000000,7.100000,0.100000
1,2024-03-01 01:00:00,7.158333,3.041667,72.666667,2.508333,234.666667,20.483333,26.366667,0.0,20.783333,97.750000,693.416667,5.091667,0.008333
2,2024-03-01 02:00:00,6.477778,2.111111,75.666667,2.433333,262.888889,21.944444,27.433333,0.0,20.033333,86.333333,616.222222,4.733333,0.011111
3,2024-03-01 03:00:00,6.045455,0.900000,74.545455,1.663636,213.000000,22.490909,27.645455,0.0,19.900000,62.818182,8692.363636,4.209091,0.136364
4,2024-03-01 04:00:00,5.430000,-0.090000,68.100000,-0.030000,256.800000,26.140000,42.440000,0.0,23.800000,38.100000,10911.800000,3.180000,0.410000


weather_forecast

In [3]:
# 데이터 불러오기
weather_forecast_1 = pd.read_csv("../../data_files/weather_forecast_1.csv")

# 'ts'를 datetime 형식으로 변환 후 1시간 단위로 맞추기
weather_forecast_1['datetime'] = pd.to_datetime(weather_forecast_1['ts'], unit='s')
weather_forecast_1['datetime'] = weather_forecast_1['datetime'].dt.round('h') + timedelta(hours=9)

# 원래 'ts' 열 제거, 'base_ts' 열 제거, 'location' 제거
weather_forecast_1.drop(columns=['ts'], inplace=True)
weather_forecast_1.drop(columns=['base_ts'], inplace=True)
weather_forecast_1.drop(columns=['location'], inplace=True)

# 1시간 단위로 그룹화하여 평균을 계산 (location을 제외한 모든 열)
weather_forecast_1 = weather_forecast_1.groupby(['datetime'], as_index=False).mean()


# 예보 데이터의 columns 순서를 actual_weather columns 순서와 맞추기
weather_forecast_1 = weather_forecast_1[['datetime', 'temp', 'real_feel_temp', 'rel_hum',
       'dew_point', 'wind_dir', 'wind_spd', 'wind_gust_spd', 'uv_idx', 'vis',
       'cld_cvr', 'ceiling', 'wet_bulb_temp', 'total_liq']]
weather_forecast_1['precip_1h'] = weather_forecast_1['total_liq']
weather_forecast_1.drop(columns=['total_liq'], inplace=True)  

# 결과 확인
weather_forecast_1.head()

,datetime,temp,real_feel_temp,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,vis,cld_cvr,ceiling,wet_bulb_temp,precip_1h
0,2024-03-02 00:00:00,1.728394,-4.320987,64.333333,-4.320987,335.555556,22.048022,50.604944,0.0,16.093400,75.555556,1700.106667,-0.555556,0.000000
1,2024-03-02 01:00:00,1.851852,-4.506173,64.333333,-4.135800,335.000000,22.048022,50.640711,0.0,15.378140,84.777778,1249.680000,-0.432099,0.056444
2,2024-03-02 02:00:00,1.790123,-5.061728,66.333333,-3.950616,334.444444,22.656000,51.481144,0.0,14.392882,89.333333,1229.360000,-0.370370,0.197556
3,2024-03-02 03:00:00,1.604940,-5.308640,67.333333,-3.888887,333.777778,23.263967,51.463267,0.0,15.195758,94.555556,1933.786667,-0.370370,0.084667
4,2024-03-02 04:00:00,1.851852,-4.876541,66.444444,-3.950616,333.888889,23.871922,50.837400,0.0,15.825200,82.555556,2600.960000,-0.308642,0.000000


elec_supply

In [4]:
# 데이터 불러오기
elec_supply = pd.read_csv("../../data_files/elec_supply.csv")

# 'ts'를 datetime 형식으로 변환 후 1시간 단위로 맞추기
elec_supply['datetime'] = pd.to_datetime(elec_supply['ts'], unit='s')
elec_supply['datetime'] = elec_supply['datetime'].dt.round('h') + timedelta(hours=9)

# 1시간 단위로 그룹화하여 평균을 계산
elec_supply = elec_supply.groupby(['datetime'], as_index=False).mean()

# 결과 확인
elec_supply.drop(columns=['ts'], inplace=True)  # 원래 'ts' 열 제거
elec_supply.head()

,datetime,공급능력(kW),현재 수요(kW),태양광 발전량kW),풍력 발전량(kW),신재생 발전량 총합(kW),공급 예비력(kW),운영 예비력(kW)
0,2024-01-01 00:00:00,1.285000e+06,757500.000000,0.0,115593.233333,123350.666667,527166.666667,328500.000000
1,2024-01-01 01:00:00,1.238308e+06,727538.461538,0.0,68802.153846,77036.569231,510692.307692,312692.307692
2,2024-01-01 02:00:00,1.214818e+06,694090.909091,0.0,45923.181818,53740.109091,520727.272727,318545.454545
3,2024-01-01 03:00:00,1.207923e+06,672000.000000,0.0,38567.161538,46614.815385,536000.000000,291000.000000
4,2024-01-01 04:00:00,1.192364e+06,666000.000000,0.0,22849.272727,30575.990909,526363.636364,318272.727273


smp_da

In [5]:
# 데이터 불러오기
smp_da = pd.read_csv("../../data_files/smp_da.csv")

# 'ts'를 datetime 형식으로 변환 후 1시간 단위로 맞추기
smp_da['datetime'] = pd.to_datetime(smp_da['ts'], unit='s') + timedelta(hours=9)

# 결과 확인
smp_da.drop(columns=['ts'], inplace=True)  # 원래 'ts' 열 제거
smp_da.head()

,하루전가격(원/kWh),datetime
0,107.39,2024-03-01 00:00:00
1,107.39,2024-03-01 01:00:00
2,95.30,2024-03-01 02:00:00
3,87.89,2024-03-01 03:00:00
4,0.00,2024-03-01 04:00:00


smp_rt_rc

In [6]:
# 데이터 불러오기
smp_rt_rc = pd.read_csv("../../data_files/smp_rt_rc.csv")

# 'ts'를 datetime 형식으로 변환 후 1시간 단위로 맞추기
smp_rt_rc['datetime'] = pd.to_datetime(smp_rt_rc['ts'], unit='s')
smp_rt_rc['datetime'] = smp_rt_rc['datetime'].dt.round('h') + timedelta(hours=9)

# 1시간 단위로 그룹화하여 평균을 계산
smp_rt_rc = smp_rt_rc.groupby(['datetime'], as_index=False).mean()

# 결과 확인
smp_rt_rc.drop(columns=['ts'], inplace=True)  # 원래 'ts' 열 제거
smp_rt_rc.head()

,datetime,실시간 임시 가격(원/kWh),실시간 확정 가격(원/kWh)
0,2024-03-01 00:00:00,95.30,95.30
1,2024-03-01 01:00:00,107.39,107.39
2,2024-03-01 02:00:00,95.30,95.30
3,2024-03-01 03:00:00,87.89,87.89
4,2024-03-01 04:00:00,86.50,86.50


##### X 데이터

- 제주 기상 실측 (n+1)
- 제주 전력 시장 실시간 (n-1)
- 제주 전력 시장 현황 (n-1)

In [7]:
actual_weather_1
smp_rt_rc['datetime'] += pd.Timedelta(days=2)
elec_supply['datetime'] += pd.Timedelta(days=2)

In [8]:
dfs = [actual_weather_1, smp_rt_rc, elec_supply]

X = reduce(lambda left, right: pd.merge(left, right, on='datetime', how='outer'), dfs)

##### Y 데이터

- 하루 전 시장 전기 예측 가격(n+1)

In [9]:
Y = smp_da

### 파이토치 모델 구성

##### 하이퍼 파라미터

In [ ]:
hidden_size = 16  # Hidden state size, you can choose other values too
num_layers = 4  # Number of LSTM layers (you can adjust this)

learning_rate = 0.001
epoches= 300

##### 날짜 설정

In [10]:
target_date = {
  # 최소 2024-03-04
  "train_start_date": '2024-03-03',
  # 최대 2024-10-22
  "train_end_date": '2024-10-22',
  # 예측할 날짜
  "predict_target_date": '2024-11-03',
}

In [11]:
X = X.loc[
  (X['datetime'] > target_date['train_start_date']) &
  (X['datetime'] <= target_date['train_end_date'])
]

In [12]:
Y = Y.loc[
  (Y['datetime'] > target_date['train_start_date']) &
  (Y['datetime'] <= target_date['train_end_date'])
]

In [13]:
from datetime import datetime, timedelta

targ = datetime.strptime(target_date['train_start_date'], '%Y-%m-%d') + timedelta(hours=1)
endd = datetime.strptime(target_date['train_end_date'], '%Y-%m-%d')

print(targ, endd)

2024-03-03 01:00:00 2024-10-22 00:00:00


In [14]:
# X 데이터의 특정 시간이 누락되었기 때문에
# Y 데이터에서도 누락된 시간대를 제거한다.

while targ <= endd:
  if not len(X.loc[X['datetime'] == targ]):
    idx = Y[Y['datetime'] == targ].index
    Y.drop(idx, inplace=True)
  
  targ = targ + timedelta(hours=1)

In [15]:
len(X), len(Y)

(5592, 5592)

##### 모듈 임포트

In [16]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.preprocessing import MinMaxScaler

In [17]:
if torch.cuda.is_available():
  print(torch.cuda.get_device_name())
  print(torch.__version__)
  print(torch.version.cuda)
  x = torch.randn(1).cuda()
  print(x)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

##### Dataset 구성

In [18]:
class ElecDataset(Dataset):
  def __init__(self, x_data, y_data):
    scaler = MinMaxScaler()

    columns_to_scale = x_data.columns[1:]
    x_data[columns_to_scale] = scaler.fit_transform(x_data[columns_to_scale])
    
    self.x_data = x_data.fillna(0)
    self.y_data = y_data.fillna(0)

  def __getitem__(self, index):
    length = 24
    target_y = self.y_data['하루전가격(원/kWh)'].iloc[index*length:(index+1)*length].to_numpy()
    targets = self.x_data.drop(columns='datetime').iloc[index*length:(index+1)*length].to_numpy()

    return torch.from_numpy(targets), torch.from_numpy(target_y)

  def __len__(self):
    return int(len(self.y_data) / 24)

In [19]:
dataset = ElecDataset(X, Y)

In [20]:
len(dataset)

233

##### dataloader 구성

In [21]:
dataloader = DataLoader(dataset=dataset,
                        batch_size=8,
                        # 데이터의 순서가 중요해서 shuffle하면 안될 듯, 그래도 해보자
                        shuffle=True,
                        drop_last=False)

##### 모델 구성

In [22]:
class LSTMModel(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, num_layers):
    super(LSTMModel, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    
    # Define the LSTM layer
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dtype=torch.float64)
    
    # Define a fully connected layer to produce output of size 1
    self.fc = nn.Linear(hidden_size, output_size, dtype=torch.float64)
  
  def forward(self, x):
    # Set initial hidden and cell states to zeros
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, dtype=torch.float64).to(x.device)  # hidden state
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, dtype=torch.float64).to(x.device)  # cell state
    
    # Forward propagate LSTM
    out, _ = self.lstm(x, (h0, c0))  # out의 shape은 (batch_size, seq_length, hidden_size)입니다.
    
    # 모든 타임스텝에 대해 최종 출력 처리
    out = self.fc(out[:, -1, :])  # 각 배치에 대해 마지막 타임스텝의 출력만 취함, out의 shape은 (batch_size, output_size)로 설정됩니다.
    
    return out

In [ ]:
input_size = len(X.columns) - 1  # Number of input features
output_size = 24  # Single output


model = LSTMModel(input_size, hidden_size, output_size, num_layers)


criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# # Example input: batch of 16 sequences, each of length 12, with 7 features per timestep
# it = iter(dataloader)
# input_tensor = next(it)[0]
# print(input_tensor)

# # Forward pass
# output = model(input_tensor)
# print(output)  # Expected output: (16, 1)

##### 학습

In [ ]:
def train(model, train_loader, criterion, optimizer, num_epochs, device):
    model = model.to(device)  # Move model to GPU/CPU

    train_history = []
    
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        
        running_loss = 0.0  # To keep track of loss
        for inputs, targets in tqdm(train_loader, ncols=100):
            # Move data to the same device as the model
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Forward pass
            outputs = model(inputs)
            
            # print(f'input: {inputs.shape}, targets: {targets.shape}, outputs: {outputs.shape}')
            # for name, param in model.named_parameters():
            #     if param.grad is not None:
            #         print(f"{name} grad: {param.grad.norm()}")

            loss = criterion(outputs, targets)

            # Backward pass and optimization
            optimizer.zero_grad()  # Clear the gradients
            loss.backward()        # Compute gradients
            # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()        # Update model parameters

            running_loss += loss.item()
            
            # print(loss.item())

        # Print the loss after each epoch
        avg_loss = running_loss / len(train_loader)
        train_history.append(avg_loss)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

    print("Training complete.")
    return model, train_history

In [ ]:
model, train_history = train(model, dataloader, criterion, optimizer, epoches, device)

100%|██████████| 30/30 [00:03<00:00,  9.51it/s]


Epoch [1/300], Loss: 18301.8086


100%|██████████| 30/30 [00:03<00:00,  8.14it/s]


Epoch [2/300], Loss: 18186.0708


100%|██████████| 30/30 [00:03<00:00,  8.94it/s]


...

Epoch [298/300], Loss: 1795.0672


100%|██████████| 30/30 [00:03<00:00,  9.03it/s]


Epoch [299/300], Loss: 1795.7289


100%|██████████| 30/30 [00:02<00:00, 10.47it/s]

Epoch [300/300], Loss: 1799.0962
Training complete.


In [26]:
torch.save(model.state_dict(), 'model2.pth')

##### 예측(미완)

In [27]:
model = LSTMModel(input_size, hidden_size, output_size, num_layers)
model.load_state_dict(torch.load('model2.pth'))

C:\Users\user\AppData\Local\Temp\ipykernel_14648\1464077963.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model2.pth'))


<All keys matched successfully>

##### X 데이터

- 제주 기상 예측 (n+1)
- 제주 전력 시장 실시간 (n-1)
- 제주 전력 시장 현황 (n-1)

In [28]:
dfs = [weather_forecast_1, smp_rt_rc, elec_supply]

input_data = reduce(lambda left, right: pd.merge(left, right, on='datetime', how='inner'), dfs)

target_day = datetime.strptime(target_date['predict_target_date'], '%Y-%m-%d')

input_data = input_data.loc[
  (input_data['datetime'] > target_day) &
  (input_data['datetime'] <= target_day + timedelta(days=1))
]

In [29]:
input_data

,datetime,temp,real_feel_temp,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,vis,...,precip_1h,실시간 임시 가격(원/kWh),실시간 확정 가격(원/kWh),공급능력(kW),현재 수요(kW),태양광 발전량kW),풍력 발전량(kW),신재생 발전량 총합(kW),공급 예비력(kW),운영 예비력(kW)
5793,2024-11-03 01:00:00,18.209867,16.358033,76.888889,14.074067,60.777778,17.327300,30.255644,0.000000,16.09340,...,0.000000,108.924000,108.924000,1.139692e+06,528153.846154,0.000000,77352.069231,77597.192308,581615.384615,364769.230769
5794,2024-11-03 02:00:00,17.716044,15.925922,77.666667,13.641967,55.222222,17.523989,29.433111,0.000000,16.09340,...,0.000000,103.236667,103.236667,1.155455e+06,511090.909091,0.000000,86352.463636,86597.045455,625363.636364,398727.272727
5795,2024-11-03 03:00:00,17.469133,15.679022,77.888889,13.580233,94.333333,17.112711,28.628433,0.000000,16.09340,...,0.000000,98.398000,98.398000,1.162154e+06,504692.307692,0.000000,92392.100000,92636.153846,647846.153846,373923.076923
5796,2024-11-03 04:00:00,17.098756,15.246933,79.333333,13.518500,22.000000,17.130600,27.805878,0.000000,16.09340,...,0.000000,95.116667,95.116667,1.188909e+06,491454.545455,0.000000,117552.909091,117796.636364,687363.636364,408000.000000
5797,2024-11-03 05:00:00,17.160489,15.246911,78.777778,13.395056,32.777778,16.486867,26.965444,0.000000,16.09340,...,0.000000,102.490000,102.490000,1.219462e+06,506769.230769,0.000000,148452.769231,148696.307692,710384.615385,424076.923077
5798,2024-11-03 06:00:00,17.037033,15.308656,78.888889,13.395056,40.666667,15.861011,25.964078,0.000000,16.09340,...,0.000000,104.500000,104.500000,1.213364e+06,524272.727273,0.000000,143401.181818,143643.363636,684363.636364,402363.636364
5799,2024-11-03 07:00:00,16.913567,15.432111,80.222222,13.456778,41.111111,15.235156,25.123656,0.000000,16.09340,...,0.000000,116.480000,116.480000,1.223154e+06,565384.615385,0.000000,151426.230769,151670.153846,653000.000000,369000.000000
5800,2024-11-03 08:00:00,18.086411,17.407400,75.000000,13.703689,37.777778,15.253033,24.515678,1.000000,16.09340,...,0.000000,137.910000,137.910000,1.237727e+06,622727.272727,0.000000,165409.000000,165729.000000,610181.818182,378909.090909
5801,2024-11-03 09:00:00,19.320978,19.074067,70.888889,13.888867,74.000000,15.449733,24.301089,1.888889,16.09340,...,0.000000,150.890000,150.890000,1.262538e+06,664461.538462,1884.663846,188450.153846,190779.846154,593153.846154,362153.846154
5802,2024-11-03 10:00:00,20.555556,21.049378,66.888889,14.197500,109.888889,14.859633,23.496422,2.777778,16.09340,...,0.000000,148.923333,148.923333,1.280000e+06,690545.454545,3222.000909,204576.636364,208088.909091,584909.090909,353909.090909


In [30]:
predict_set = ElecDataset(input_data, Y.iloc[:24])

predict_dataloader = DataLoader(dataset=predict_set,
                        batch_size=24,
                        shuffle=False)

In [31]:
model.eval()

with torch.no_grad():
  for inputs, _ in predict_dataloader:
    inputs = inputs.to(device)
    predictions = model(inputs)

print(predictions)

tensor([[119.0990, 110.7050, 107.0808, 104.3667, 105.2238, 106.8239, 113.4568,
         118.8413, 119.4623, 115.4636, 112.9813, 103.9451,  97.9438, 106.8326,
         117.6439, 120.0139, 125.3977, 127.4577, 130.2435, 129.8115, 129.1830,
         127.8369, 126.3316, 124.2108]], dtype=torch.float64)


In [32]:
flattened_list = predictions.view(-1).tolist()

print(flattened_list)

[119.09903126717316, 110.70499435286814, 107.08081382600454, 104.36674603261201, 105.22375314834818, 106.82385367398469, 113.45677060934172, 118.84125131529626, 119.46231122808909, 115.46358763286263, 112.98126100764735, 103.94508614145576, 97.94380976146962, 106.8326359560034, 117.64390801755789, 120.01388857722434, 125.39767528443559, 127.45771438191075, 130.243456847862, 129.81147185267702, 129.1830274192109, 127.83691391564821, 126.33156495642119, 124.210770761254]
